In [1]:
exec('from __future__ import unicode_literals')

import os
import sys
import random

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from string import punctuation
import onmt.keyphrase.pke as pke
from nltk.corpus import stopwords

import onmt.keyphrase.kp_inference as kp_inference

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [37]:
import importlib
importlib.reload(kp_inference)

<module 'onmt.keyphrase.kp_inference' from '/Users/memray/project/kp/OpenNMT-kpg/onmt/keyphrase/kp_inference.py'>

### Load a paragraph from 10-K docs (assume current directory is OpenNMT-kpg/notebook/)

In [4]:
data_root_path = '../data/salesforce/10k'
print(os.path.abspath(data_root_path))
doc_dicts = []
for subdir, dirs, files in os.walk(data_root_path):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
#             print(filepath)
            text = open(filepath, 'r').readlines()
            pars = [l for l in text if len(l.split()) > 20]
            doc = {'name': file, 'path': filepath, 'text': text, 'paragraph': pars}
#             print('#line=%d, #par=%d' % (len(text), len(pars)))
            doc_dicts.append(doc)
    
print('Loaded #(10k docs)=%d' % (len(doc_dicts)))

/Users/memray/project/kp/OpenNMT-kpg/data/salesforce/10k
Loaded #(10k docs)=11252


##### Sample a paragraph

In [12]:
doc_id = random.randint(0, len(doc_dicts))
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par_id = random.randint(0, len(doc_dicts[doc_id]['paragraph']))
text_to_extract = pars[par_id]
print(doc_id)
print(par_id)
print(text_to_extract)

7705
3
CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.



### Or load a PROD description

In [2]:
PROD_path = '../data/salesforce/SF_Prod'

prod_dicts = []
for subdir, dirs, files in os.walk(PROD_path):
    for file in files:
        filepath = subdir + os.sep + file
        text = open(filepath, 'r').readlines()
        text = '\n'.join([l.strip() for l in text])
        doc = {'name': file, 'path': filepath, 'text': text}
        prod_dicts.append(doc)

print('Loaded #(PROD docs)=%d' % (len(prod_dicts)))

doc_id = random.randint(0, len(prod_dicts))
doc = prod_dicts[doc_id]
text_to_extract = doc['text']
print(doc_id)
print(doc['name'])
print(text_to_extract)

Loaded #(PROD docs)=20
2
Marketing_norm
enables and know across personalize engage analyze
Deliver experiences personalized step campaign every lifecycle customer
and manage experiences customer management driving engagement valuable interaction right real-time
and transparent create customer valuable reach Expand marketers 2nd sharing data share and activate data Capture and any source audiences platform publishers then party can data trusted
Use data build department every email smarter campaigns marketing sophisticated basic
and Send consistent SMS, chat messages
and engage create customer
Connect artificial social and service powered platform
actionable reports drive and Enable intelligence marketing data unifying AI-powered visualizing creating
and partners sell consumers and Enable
and known data Connect across sources and gain view unified devices multiple unknown
Create business experiences customer seamless connecting and channels stage every lifecycle
and return maximize impa

### Supervised Deep Keyphrase Model

In [3]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = build_translator(opt, report_score=False)

In [6]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


Translating 10/1
Total translation time (s): 0.344692
Average translation time (s): 0.344692
Tokens per second: 5.802278
Paragraph:
	enables and know across personalize engage analyze
Deliver experiences personalized step campaign every lifecycle customer
and manage experiences customer management driving engagement valuable interaction right real-time
and transparent create customer valuable reach Expand marketers 2nd sharing data share and activate data Capture and any source audiences platform publishers then party can data trusted
Use data build department every email smarter campaigns marketing sophisticated basic
and Send consistent SMS, chat messages
and engage create customer
Connect artificial social and service powered platform
actionable reports drive and Enable intelligence marketing data unifying AI-powered visualizing creating
and partners sell consumers and Enable
and known data Connect across sources and gain view unified devices multiple unknown
Create business experie

### PKE models

#### TF-IDF

In [7]:
dataset_name = 'SF_Prod'
dataset_path = '../data/salesforce/%s/' % dataset_name
_ = kp_inference.extract_pke(text_to_extract, method='tfidf' , dataset_path=dataset_path,
            df_path=os.path.abspath(dataset_path + '../%s.df.tsv.gz' % dataset_name), top_k=20)

	1: enables (0.0000)
	2: enables and (0.0000)
	3: enables and know (0.0000)
	4: and (0.0000)
	5: and know (0.0000)
	6: and know across (0.0000)
	7: know (0.0000)
	8: know across (0.0000)
	9: know across personalize (0.0000)
	10: across (0.0000)
	11: across personalize (0.0000)
	12: across personalize engage (0.0000)
	13: personalize (0.0000)
	14: personalize engage (0.0000)
	15: personalize engage analyze (0.0000)
	16: engage (0.0000)
	17: engage analyze (0.0000)
	18: analyze (0.0000)
	19: deliver (0.0000)
	20: deliver experiences (0.0000)


#### YAKE

In [8]:
_ = kp_inference.extract_pke(text_to_extract, method='yake', top_k=20)

	1: send consistent sms (0.0001)
	2: data connect across (0.0002)
	3: use data build (0.0002)
	4: known data connect (0.0002)
	5: deliver experiences personalized (0.0002)
	6: enable intelligence marketing (0.0002)
	7: activate data capture (0.0002)
	8: create customer valuable (0.0002)
	9: engage create customer (0.0002)
	10: connect across sources (0.0002)
	11: transparent create customer (0.0002)
	12: valuable reach expand (0.0002)
	13: connect artificial social (0.0002)
	14: create business experiences (0.0002)
	15: measure optimize performance (0.0002)
	16: reach expand marketers (0.0002)
	17: expand marketers 2nd (0.0002)
	18: every lifecycle customer (0.0002)
	19: manage experiences customer (0.0003)
	20: experiences customer management (0.0003)


#### TextRank

In [9]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: performance marketing total investment (0.0812)
	2: marketing data (0.0756)
	3: view unified devices multiple (0.0734)
	4: customer valuable (0.0709)
	5: experiences customer (0.0639)
	6: data build (0.0621)
	7: valuable interaction right (0.0620)
	8: data connect (0.0615)
	9: data (0.0454)
	10: customer (0.0421)
